In [44]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [45]:
tsp_data_txt = open('tsp_dataset.txt')

In [46]:
lines = tsp_data_txt.readlines()

In [47]:
tsp_data_txt = open('tsp_dataset.txt')
data = {}
lines = tsp_data_txt.readlines()
distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['reward_matrix'] = [1 for i  in range(0,len(distance_matrix))]
data['depot'] = 0
data['n_state'] = sum(list(range(1,len(distance_matrix))))
data['n_action'] = len(distance_matrix)
data['visited_nodes'] = []


In [48]:
for i in range(0,len(data['distance_matrix'])):
    data['distance_matrix'][i][i] = 9999


In [49]:
def change_reward(envp):
    for i in range(0,envp['n_state']):
        for j in range(0,envp['n_action']):
            for k in range(0,envp['n_action']):
                if k==0:
                    env['P'][i][j][k][2]= 10
                    env['P'][i][j][k][3]= True
                    break
    return envp

In [50]:
def initialze_value(env):
    initial_P = 1/(env['n_action']-1)
    env['P'] = {}
    
    #P[state][action] = [(prob, next state, reward, done), ...]
    
    for i in range(0,env['n_state']):
        env['P'][i] ={}
        
        if(i==104):
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                initial_P = 1/(env['n_action']-1)
                for k in range(0,env['n_action']):
                    if (j == env['depot'] and k == env['depot']):
                        ini_temp = [1,k,10,True]
                        env['P'][i][j].append(ini_temp)
                        initial_P = 0
                        continue
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
        else:
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                for k in range(0,env['n_action']):
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
            
            
    return env
    

In [51]:
env = initialze_value(data)

In [52]:
class Double_Q_learning():
    def __init__(self,n_state,n_action,alpha=0.5,epsilon=1.0,gamma=0.999):
        self.n_state = n_state
        self.n_action = n_action
        self.alpha_init = alpha
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma
        
        # Initialize Two Q tables
        self.Q_A = np.zeros([n_state,n_action])
        self.Q_B = np.zeros([n_state,n_action])
        
    def update_Q(self,state,action,reward,state_prime,done):
        """
        Update value
        """
        if np.random.uniform() < 0.5:
            Q_update = self.Q_A
            Q_target = self.Q_B
        else:
            Q_update = self.Q_B
            Q_target = self.Q_A
        action_prime = np.argmax(Q_update[state_prime])
        

        # TD target
        if done:
            td_target = reward
        else:
            td_target = reward + self.gamma * Q_target[state_prime][action_prime]

        Q_old = Q_update[state][action]

        td_error = td_target - Q_old # TD error
        Q_update[state,action] = Q_old + self.alpha * td_error # update Q
        
    def update_epsilon(self,epsilon):
        self.epsilon = np.min([epsilon,1.0]) 
        
    def update_alpha(self,alpha):
        self.alpha = np.min([alpha,self.alpha_init]) 
        
    def get_action(self,state):
        """
        Get action
        """
        if np.random.uniform() < self.epsilon:
            action = np.random.randint(0,high=self.n_action)
        else:
            action = np.argmax(self.Q_A[state])
        return action 


In [53]:
n_state = env['n_state']
n_action = env['n_action']
agent = Double_Q_learning(n_state,n_action,epsilon=1.0,alpha=0.1,gamma=0.999)

In [54]:
n_episiode = 1000
ini_env = env
for e_idx in range(n_episiode):
    env['visited_nodes'] = []
    env = ini_env
    state = env['depot']
    env['visited_nodes'].append(state)
    action = agent.get_action(state)
    done = False
    while not done:
        info,state_prime, reward, done = env['P'][state][action][action]
        action_prime = agent.get_action(state_prime) # Get next action
        agent.update_Q(state, action, reward, state_prime, done) # learns Q
        state = state_prime
        
        action = action_prime
        if action not in env['visited_nodes']:
            env['visited_nodes'].append(state)
        if len(env['visited_nodes'])==15:
            done=True
                
    agent.update_epsilon(100/(e_idx+1)) # Decaying epsilon
print ("Training Done.")

Training Done.


In [56]:
# Test
gamma = 0.99
env = ini_env
ret = 0
state = env['depot']
visited_list = []
for tick in range(1000):
    print("\n tick:[{}]".format(tick))
    print(visited_list)
    action = agent.get_action(state) # select action
    if len(visited_list) !=14 and action==0:
        continue
    if action not in  visited_list:
        visited_list.append(action)
    _,next_obs,reward,done = env['P'][state][action][action]
    obs = next_obs+state
    ret = reward + gamma*ret 
    state = next_obs
    
    if len(visited_list)== 15: break
print ("Return is [{:.3f}]".format(ret))


 tick:[0]
[]

 tick:[1]
[]

 tick:[2]
[]

 tick:[3]
[]

 tick:[4]
[]

 tick:[5]
[]

 tick:[6]
[]

 tick:[7]
[]

 tick:[8]
[]

 tick:[9]
[]

 tick:[10]
[]

 tick:[11]
[]

 tick:[12]
[6]

 tick:[13]
[6, 4]

 tick:[14]
[6, 4]

 tick:[15]
[6, 4]

 tick:[16]
[6, 4]

 tick:[17]
[6, 4]

 tick:[18]
[6, 4]

 tick:[19]
[6, 4]

 tick:[20]
[6, 4]

 tick:[21]
[6, 4]

 tick:[22]
[6, 4]

 tick:[23]
[6, 4]

 tick:[24]
[6, 4]

 tick:[25]
[6, 4]

 tick:[26]
[6, 4]

 tick:[27]
[6, 4]

 tick:[28]
[6, 4]

 tick:[29]
[6, 4]

 tick:[30]
[6, 4]

 tick:[31]
[6, 4]

 tick:[32]
[6, 4]

 tick:[33]
[6, 4]

 tick:[34]
[6, 4]

 tick:[35]
[6, 4, 3]

 tick:[36]
[6, 4, 3]

 tick:[37]
[6, 4, 3]

 tick:[38]
[6, 4, 3]

 tick:[39]
[6, 4, 3]

 tick:[40]
[6, 4, 3]

 tick:[41]
[6, 4, 3]

 tick:[42]
[6, 4, 3]

 tick:[43]
[6, 4, 3]

 tick:[44]
[6, 4, 3]

 tick:[45]
[6, 4, 3]

 tick:[46]
[6, 4, 3]

 tick:[47]
[6, 4, 3]

 tick:[48]
[6, 4, 3]

 tick:[49]
[6, 4, 3, 1]

 tick:[50]
[6, 4, 3, 1, 2]

 tick:[51]
[6, 4, 3, 1, 2]

 tick:

In [57]:
def get_reward(visit_list):
    depot =0
    distance = (data['distance_matrix'])[0][visit_list[0]]
    prev_cite = visit_list[0]
    for i in range(1,14):
        distance+= (data['distance_matrix'])[prev_cite][visit_list[i]]
        prev_cite =visit_list[i]

    distance+= (data['distance_matrix'])[prev_cite][visit_list[14]]
    return distance

In [58]:
get_reward([6, 4, 3, 1, 2, 8, 7, 13, 5, 9, 10, 11, 12, 14,0])

689.0